# Visualization with Streamlit

In this step we will make use of streamlit library to build the plots of predictions created using regression and nbeats time series.

In [33]:
%%writefile 'src/streamlit-app.py'

import streamlit as st
import digitalhub as dh
import pandas as pd

PROJECT_NAME = "parcheggi-nk-scheduler-digitalhubdev"
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))

prediction_nbeats = proj.get_dataitem("parking_prediction_nbeats_model")
prediction_sarimax_regression = proj.get_dataitem("parking_data_predicted_regression")

rdf1 = prediction_sarimax_regression.as_df();

rdf2 = prediction_nbeats.as_df()
rdf2.datetime = pd.to_datetime(rdf2.datetime, unit='ms')

st.write("""prediction Sarimax regression""")
st.line_chart(rdf1, x="datetime", y="predicted_mean")

st.write("""prediction Nbeats model""")
st.line_chart(rdf2, x="datetime", y="predicted_mean")


Overwriting src/streamlit-app.py


In [34]:
#!streamlit run src/streamlit-app.py --browser.gatherUsageStats false

In [35]:
import digitalhub as dh
import getpass as gt

PROJECT_NAME = "parcheggi-nk-scheduler-"+gt.getuser()
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))

created project parcheggi-nk-scheduler-digitalhubdev


In [36]:
func = proj.new_function(name="streamlit-viz-v3",
                         kind="container",
                         python_version="PYTHON3_10",
                         lang="python",
                         base_image="ghcr.io/scc-digitalhub/digitalhub-serverless/python-runtime:3.9-latest",
                         image="ghcr.io/scc-digitalhub/digitalhub-serverless/python-runtime:3.9-latest",
                         code_src="src/streamlit-app.py", args= ["run", "src/streamlit-app.py"], command= "streamlit")

In [37]:
# func._tasks

In [38]:
func.run(action="build", instructions=["pip install streamlit"])

{'kind': 'container+run', 'metadata': {'project': 'parcheggi-nk-scheduler-digitalhubdev', 'name': '9713ff68-55ae-4a99-a79a-e65c63322e7c', 'created': '2024-10-14T10:37:45.446Z', 'updated': '2024-10-14T10:37:45.459Z', 'created_by': 'digitalhubdev', 'updated_by': 'digitalhubdev'}, 'spec': {'task': 'container+build://parcheggi-nk-scheduler-digitalhubdev/streamlit-viz-v3:227179bc-47df-42c1-ae58-1e637d4969ab', 'local_execution': False, 'function': 'container://parcheggi-nk-scheduler-digitalhubdev/streamlit-viz-v3:227179bc-47df-42c1-ae58-1e637d4969ab', 'source': {'source': 'src/streamlit-app.py', 'base64': 'CmltcG9ydCBzdHJlYW1saXQgYXMgc3QKaW1wb3J0IGRpZ2l0YWxodWIgYXMgZGgKaW1wb3J0IHBhbmRhcyBhcyBwZAoKUFJPSkVDVF9OQU1FID0gInBhcmNoZWdnaS1uay1zY2hlZHVsZXItZGlnaXRhbGh1YmRldiIKcHJvaiA9IGRoLmdldF9vcl9jcmVhdGVfcHJvamVjdChQUk9KRUNUX05BTUUpCnByaW50KCJjcmVhdGVkIHByb2plY3Qge30iLmZvcm1hdChQUk9KRUNUX05BTUUpKQoKcHJlZGljdGlvbl9uYmVhdHMgPSBwcm9qLmdldF9kYXRhaXRlbSgicGFya2luZ19wcmVkaWN0aW9uX25iZWF0c19tb2RlbCIpCnBy

In [39]:
# func.spec.to_dict()

In [40]:
#fun.run(action="serve", command='streamlit run src/streamlit-app.py --browser.gatherUsageStats false', )

In [41]:
func.run(action="serve", service_ports=[{"port": 5002, "target_port": 8501}])
#service_type = "NodePort"

{'kind': 'container+run', 'metadata': {'project': 'parcheggi-nk-scheduler-digitalhubdev', 'name': '78f36d4c-e954-4200-a639-fdb66f6cc3cc', 'created': '2024-10-14T10:40:14.845Z', 'updated': '2024-10-14T10:40:14.858Z', 'created_by': 'digitalhubdev', 'updated_by': 'digitalhubdev'}, 'spec': {'task': 'container+serve://parcheggi-nk-scheduler-digitalhubdev/streamlit-viz-v3:227179bc-47df-42c1-ae58-1e637d4969ab', 'local_execution': False, 'function': 'container://parcheggi-nk-scheduler-digitalhubdev/streamlit-viz-v3:227179bc-47df-42c1-ae58-1e637d4969ab', 'source': {'source': 'src/streamlit-app.py', 'base64': 'CmltcG9ydCBzdHJlYW1saXQgYXMgc3QKaW1wb3J0IGRpZ2l0YWxodWIgYXMgZGgKaW1wb3J0IHBhbmRhcyBhcyBwZAoKUFJPSkVDVF9OQU1FID0gInBhcmNoZWdnaS1uay1zY2hlZHVsZXItZGlnaXRhbGh1YmRldiIKcHJvaiA9IGRoLmdldF9vcl9jcmVhdGVfcHJvamVjdChQUk9KRUNUX05BTUUpCnByaW50KCJjcmVhdGVkIHByb2plY3Qge30iLmZvcm1hdChQUk9KRUNUX05BTUUpKQoKcHJlZGljdGlvbl9uYmVhdHMgPSBwcm9qLmdldF9kYXRhaXRlbSgicGFya2luZ19wcmVkaWN0aW9uX25iZWF0c19tb2RlbCIpCnBy

## Expose the service

To make it accessible from outside, we'll need to create an API gateway.

Go to the Kubernetes Resource Manager component (available from dashboard) and go to the API Gateways section. To expose a service it is necessary to define

name of the gateway
the service to expose
the endpoint where to publish
and the authentication method (right now only no authentication or basic authentication are available). in case of basic authentication it is necessary to specify Username and Password.
The platform by default support exposing the methods at the subdomains of services.<platform-domain>, where platform-domain is the domain of the platform instance.

<img src="streamlit.jpg">

Save and, after a few moments, you will be able to call the API at the address you defined! If you set Authentication to Basic, don't forget that you have to provide the credentials.